In [21]:
import pandas as pd
from db_connect import EngineConnect as DatabaseConnect

import config

In [45]:
class PnlExecutor:
    def __init__(
        self,
        *kwargs,
        environment: str,
        schema: str,
        table: str,
        db: DatabaseConnect,
    ):
        self.environment = environment
        self.schema = schema
        self.table = table
        self.db = db
        self.target_pnl_table = 'pnl_target'
        self.pre_schema = config.DWH_PRE_DATA_SCHEMA
        
    def target_pnl(self):
        df = pd.read_excel(r'./backup/pnl_target.xlsx')
        col_rename = {
            'Year': 'year',
            'Project': 'project_name',
            'Target REVENUE': 'targeted_revenue',
            '% Target_EBIT\n(% EBIT/Revenue)': 'targeted_ebit_revenue',
            'Sub-team': 'sub_team',
        }
        df = df.rename(columns = col_rename)
        df['date_key'] = [int(str(item) +'0101') for item in df['year']] 
        self.db.create_df(df, self.pre_schema, self.target_pnl_table)

        
        
       
        
        
    def summary_pnl(self):
        df = pd.read_excel (r'./backup/pnl.xlsx')
        col_rename = {
            'Month': 'month',
            'Year': 'year',
            'Mon_year': 'date_key',
            'Project name': 'project_name',
            'Customer': 'customer',
            'FTE (DPOs/IPOs/EPOs)': 'fte',
            'Actual working time (hour)': 'actual_working_time',
            'Free capacity in project (hour)': 'free_capacity_in_project_hour',
            'Non working time (hour)': 'non_working_time_hour',
            'TOTAL REVENUE': 'total_revenue',
            '% Distribution/Total revenue': 'distribution_over_revenue',
            'TOTAL VARIABLE COST': 'total_variable_cost',
            'Personnel cost': 'personnel_cost',
            '%/Production cost': 'production_cost_percent',
            'DPOs/IPOs/EPOs': 'dpo_ipo_epo',
            'Salary (labor contract)': 'labor_contract_salary',
            'Digipay salary': 'digipay_salary',
            'Needed training cost': 'training_cost',
            'Salary (service & others)': 'service_salary',
            'Salary (probation)': 'probation_salary',
            'Overtime expenses': 'overtime_expense',
            '3rd shift Allowance': 'third_shift_allowance',
            'Severnace allowance': 'severnance_allowance',
            'Other Allowance': 'other_allowance',
            ' 13th salary': 'thirteenth_allowance',
            'Social Insurance (21,5%)': 'social_insurance',
            'Union fee (2%)': 'union_fee',
            'Directly Bonus': 'directly_bonus',
            'PJM': 'pjm',
            'Software& Tester': 'software_tester',
            'QMD': 'qmd',
            'Free capacity in project': 'free_capacity_in_project_cost',
            'Cost for non-working time': 'non_working_time_cost',
            'Other Bonus (healthcheck, company trip, outing trip)': 'other_bonus',
            'Business trip': 'business_trip',
            'Selling & Marketing expenses': 'selling_marketing_expenses',
            'Oursourcing': 'oursourcing',
            'Internet line (special projects online)': 'internet_line',
            'CONTRIBUTION': 'contribution',
            'Operating cost': 'operating_cost',
            '% GC/Revenue': 'gc_revenue_percent',
            'Fixed cost (3.1+3.2+3.3+3.4):': 'fixed_cost',
            'Operating cost': 'operating_cost',
            'Depreciation fix asset + Tool & small equipment.': 'depreciation',
            'Provision': 'provision',
            'Administration & General expenses (back office)': 'administration_general_cost',
            'EBIT': 'ebit',
            'Segment': 'segment',
            'Team name': 'team_name',
            'PJM code': 'pjm_code'
        }
        col_keep = [
            'month', 'year', 'date_key', 'project_name', 'customer', 'fte','actual_working_time', 'free_capacity_in_project_hour', 'non_working_time_hour', 
            'total_revenue', 'distribution_over_revenue', 'total_variable_cost', 'personnel_cost', 'production_cost_percent', 'dpo_ipo_epo', 
            'labor_contract_salary', 'digipay_salary', 'training_cost', 'service_salary', 'probation_salary','overtime_expense', 'third_shift_allowance',
            'severnance_allowance', 'other_allowance', 'thirteenth_allowance', 'social_insurance', 'union_fee', 'directly_bonus', 'pjm', 'software_tester',
            'qmd', 'free_capacity_in_project_cost', 'non_working_time_cost', 'other_bonus', 'business_trip', 'selling_marketing_expenses', 'oursourcing',
            'internet_line', 'contribution', 'gc_revenue_percent', 'fixed_cost', 'operating_cost', 'depreciation', 'provision','administration_general_cost', 
            'ebit', 'segment', 'team_name', 'pjm_code', 
        ]
        df = df.rename(columns = col_rename)[col_keep]
        df['month'] = [int(str(item).split('.')[0]) for item in df['month']]
        date_key = []
        for i in range(len(df['month'])):
            if int(df['month'][i]) < 10:
                month = '0'+ str(df['month'][i])
            else:
                month = str(df['month'][i])
            date_key.append(int(str(df['year'][i]) + month + '01'))
        df['date_key'] = date_key
        self.db.create_df(df, self.schema, self.table)

In [47]:
db_connect = DatabaseConnect(uri = config.DWH_SQLALCHEMY_URI)
executor = PnlExecutor(
    environment = config.ENVIRONMENT,
    schema = config.DWH_MART_PNL_SCHEMA,
    table = config.DWH_MART_PNL_TABLE,
    db = db_connect,
)
# executor.summary_pnl()
executor.target_pnl()